<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b9ed0482774896b2f7914f23a92ba3c1fa29eb079aa9886a71f4de739cd17e23
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-30 12:59:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 7.41 K (0.05%)
Current PnL: -23.04 L (-15.5%)
CY Booked + Current PnL: -11.60 L (-7.8%)
-------------------
Total profit:  1.10 L
Total loss:  -24.15 L
-------------------
Total Booked + Current PnL: 15.38 L (12.64%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.38%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.91 L (65.11%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112440.0,11936.0,17170.0,4.04,11.88,15.27,28.96,79.0,0.70,0.82,49.88,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.81,38.0,M-SC,2.46,83532.0,-17245.0,17341.0,0.43,-17.11,20.76,0.09,245.0,-0.99,0.61,9.74,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.42,188990.0,24320.0,19012.0,-0.30,14.77,10.06,26.31,89.0,1.28,1.38,23.93,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,48.0,H-MC,2.88,205785.0,14043.0,23706.0,2.37,7.32,11.52,19.69,99.0,0.59,1.51,14.62,XY25,NTT,AC
42,ITC,452.00,-39.19,43.0,H-LC,1.95,196798.0,-3340.0,24226.0,-1.18,-1.67,12.31,10.44,4.0,-0.14,1.44,3.80,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181621.0,478.0,44988.0,6.05,0.26,24.77,25.10,235.0,0.01,1.33,45.55,XY24,NTT,HEALTHCARE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112440.0,11936.0,17170.0,4.04,11.88,15.27,28.96,79.0,0.70,0.82,49.88,MH,ATH,METALS
79,UNIONBANK,163.00,-13.18,57.0,M-LC,7.13,157600.0,16760.0,27738.0,3.38,11.90,17.60,31.59,66.0,0.60,1.15,41.54,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.90,188528.0,8720.0,103690.0,3.21,4.85,55.00,62.52,88.0,0.08,1.38,40.59,XR,NTT,BANKS
32,HINDZINC,730.22,27.88,64.0,M-LC,3.40,208408.0,3332.0,109227.0,2.78,1.62,52.41,54.89,52.0,0.03,1.53,26.11,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.1,-45.66,20.0,H-SC,14.27,82210.0,-20290.0,60893.0,-3.87,-19.80,74.07,39.61,114.0,-0.33,0.60,0.00,AR,ATH,MISC
85,WHIRLPOOL,2270.0,-47.10,25.0,M-SC,1.26,89310.0,-2188.0,80942.0,-3.39,-2.39,90.63,86.07,223.0,-0.03,0.65,29.69,XR,NTT,DURABLES
38,INDIGOPNTS,1408.0,99.78,27.0,M-SC,1.55,130981.0,-43578.0,43617.0,-3.15,-24.96,33.30,0.02,221.0,-1.00,0.96,13.93,OX40N,NTT,PAINTS
81,VAIBHAVGBL,521.0,46.56,43.0,H-SC,5.81,127143.0,-55632.0,166697.0,-3.03,-30.44,131.11,60.77,125.0,-0.33,0.93,13.88,XR,NTT,JEWELLERY
26,GREENPANEL,537.0,193.93,39.0,M-SC,0.99,131126.0,-51952.0,130929.0,-2.64,-28.38,99.85,43.14,230.0,-0.40,0.96,23.34,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181621.0,478.0,44988.0,6.05,0.26,24.77,25.10,235.0,0.01,1.33,45.55,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-55.57,24.0,H-SC,9.97,124184.0,848.0,132914.0,-1.00,0.69,107.03,108.45,119.0,0.01,0.91,23.19,AR,ATH,MISC
35,ICICIPRULI,790.00,-21.68,44.0,H-MC,2.20,134752.0,-1036.0,43781.0,1.84,-0.76,32.49,31.48,107.0,-0.02,0.99,11.26,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,96.03,55.0,H-SC,10.79,124038.0,-18441.0,37968.0,-0.30,-12.94,30.61,13.70,163.0,-0.49,0.91,46.40,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,38.0,M-SC,2.46,83532.0,-17245.0,17341.0,0.43,-17.11,20.76,0.09,245.0,-0.99,0.61,9.74,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220518.0,-49149.0,85473.0,-0.63,-18.23,38.76,13.47,138.0,-0.58,1.62,11.53,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,37.0,H-SC,1.59,141450.0,-34453.0,76482.0,-0.14,-19.59,54.07,23.89,149.0,-0.45,1.04,22.16,OX40N,NTT,CERAMICS
66,SIS,528.0,2069.95,38.0,H-SC,4.01,87580.0,-23452.0,47057.0,-0.51,-21.12,53.73,21.26,156.0,-0.50,0.64,18.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181621.0,478.0,44988.0,6.05,0.26,24.77,25.10,235.0,0.01,1.33,45.55,XY24,NTT,HEALTHCARE
60,SAIL,228.00,45.95,58.0,M-MC,11.20,230748.0,5786.0,161870.0,1.74,2.57,70.15,74.53,192.0,0.04,1.69,35.65,XY24,BTT,STEEL
32,HINDZINC,730.22,27.88,64.0,M-LC,3.40,208408.0,3332.0,109227.0,2.78,1.62,52.41,54.89,52.0,0.03,1.53,26.11,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.42,188990.0,24320.0,19012.0,-0.30,14.77,10.06,26.31,89.0,1.28,1.38,23.93,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,48.0,H-MC,2.88,205785.0,14043.0,23706.0,2.37,7.32,11.52,19.69,99.0,0.59,1.51,14.62,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112440.0,11936.0,17170.0,4.04,11.88,15.27,28.96,79.0,0.70,0.82,49.88,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.90,188528.0,8720.0,103690.0,3.21,4.85,55.00,62.52,88.0,0.08,1.38,40.59,XR,NTT,BANKS
32,HINDZINC,730.22,27.88,64.0,M-LC,3.40,208408.0,3332.0,109227.0,2.78,1.62,52.41,54.89,52.0,0.03,1.53,26.11,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.57,24.0,H-SC,9.97,124184.0,848.0,132914.0,-1.00,0.69,107.03,108.45,119.0,0.01,0.91,23.19,AR,ATH,MISC
86,WIPRO,420.00,-15.15,40.0,M-LC,5.58,149153.0,-1792.0,111253.0,0.39,-1.19,74.59,72.51,53.0,-0.02,1.09,4.76,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.82,26.0,H-LC,13.02,253141.0,-14224.0,79689.0,0.05,-5.32,31.48,24.48,48.0,-0.18,1.85,46.73,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.14,206351.0,-45417.0,98636.0,0.10,-18.04,47.80,21.13,27.0,-0.46,1.51,11.00,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,30.0,H-LC,12.28,273098.0,-72858.0,147719.0,0.32,-21.06,54.09,21.64,1.0,-0.49,2.00,0.32,X40,ATH,IT
83,VBL,671.64,-21.10,33.0,H-LC,8.82,283974.0,-31868.0,144543.0,-1.46,-10.09,50.90,35.67,5.0,-0.22,2.08,2.37,X40N,ATH,FMCG
19,CIPLA,1795.00,-23.21,37.0,H-LC,3.97,204842.0,342.0,41071.0,0.48,0.17,20.05,20.25,10.0,0.01,1.50,8.92,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,42.0,H-LC,1.60,229815.0,-16336.0,95534.0,-0.99,-6.64,41.57,32.18,16.0,-0.17,1.68,27.01,X200,ATH,IT
42,ITC,452.0,-39.19,43.0,H-LC,1.95,196798.0,-3340.0,24226.0,-1.18,-1.67,12.31,10.44,4.0,-0.14,1.44,3.80,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.25,42.0,H-LC,2.40,155550.0,-30545.0,78024.0,0.47,-16.41,50.16,25.51,15.0,-0.39,1.14,14.63,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,40.0,H-LC,3.76,213049.0,2683.0,26099.0,-0.52,1.28,12.25,13.68,37.0,0.10,1.56,18.01,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.21,37.0,H-LC,3.97,204842.0,342.0,41071.0,0.48,0.17,20.05,20.25,10.0,0.01,1.50,8.92,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.28,273098.0,-72858.0,147719.0,0.32,-21.06,54.09,21.64,1.0,-0.49,2.00,0.32,X40,ATH,IT
80,UNITDSPR,1644.00,-14.80,43.0,H-LC,7.21,223256.0,-10888.0,59520.0,-0.80,-4.65,26.66,20.77,86.0,-0.18,1.64,1.95,X40N,NTT,BREWERIES
83,VBL,671.64,-21.10,33.0,H-LC,8.82,283974.0,-31868.0,144543.0,-1.46,-10.09,50.90,35.67,5.0,-0.22,2.08,2.37,X40N,ATH,FMCG
42,ITC,452.00,-39.19,43.0,H-LC,1.95,196798.0,-3340.0,24226.0,-1.18,-1.67,12.31,10.44,4.0,-0.14,1.44,3.80,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.66,38.0,H-LC,10.12,220067.0,-21857.0,85122.0,1.03,-9.03,38.68,26.15,8.0,-0.26,1.61,3.99,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.25,42.0,H-LC,2.40,155550.0,-30545.0,78024.0,0.47,-16.41,50.16,25.51,15.0,-0.39,1.14,14.63,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,43.0,H-LC,1.95,196798.0,-3340.0,24226.0,-1.18,-1.67,12.31,10.44,4.0,-0.14,1.44,3.80,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,37.0,H-LC,3.97,204842.0,342.0,41071.0,0.48,0.17,20.05,20.25,10.0,0.01,1.50,8.92,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.14,206351.0,-45417.0,98636.0,0.10,-18.04,47.80,21.13,27.0,-0.46,1.51,11.00,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,40.0,H-LC,3.76,213049.0,2683.0,26099.0,-0.52,1.28,12.25,13.68,37.0,0.10,1.56,18.01,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.28,273098.0,-72858.0,147719.0,0.32,-21.06,54.09,21.64,1.0,-0.49,2.00,0.32,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.20,307678.0,-5210.0,176884.0,0.19,-1.67,57.49,54.87,3.0,-0.03,2.25,4.87,X40,BTT,IT
42,ITC,452.00,-39.19,43.0,H-LC,1.95,196798.0,-3340.0,24226.0,-1.18,-1.67,12.31,10.44,4.0,-0.14,1.44,3.80,X40,NTT,FMCG
83,VBL,671.64,-21.10,33.0,H-LC,8.82,283974.0,-31868.0,144543.0,-1.46,-10.09,50.90,35.67,5.0,-0.22,2.08,2.37,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,46.0,H-LC,7.69,249619.0,-12000.0,131225.0,0.52,-4.59,52.57,45.57,7.0,-0.09,1.83,6.22,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,40.25,78432.0,-35117.0,75169.0,1.04,-30.93,95.84,35.27,268.0,-0.47,0.57,91.96,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.28,89985.0,-7995.0,29596.0,-0.89,-8.16,32.89,22.05,152.0,-0.27,0.66,30.80,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,7.04,92898.0,-9312.0,102699.0,0.36,-9.11,110.55,91.37,208.0,-0.09,0.68,50.80,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112440.0,11936.0,17170.0,4.04,11.88,15.27,28.96,79.0,0.70,0.82,49.88,MH,ATH,METALS
78,UJJIVANSFB,60.00,96.03,55.0,H-SC,10.79,124038.0,-18441.0,37968.0,-0.30,-12.94,30.61,13.70,163.0,-0.49,0.91,46.40,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,69.0,M-SC,14.24,181621.0,478.0,44988.0,6.05,0.26,24.77,25.10,235.0,0.01,1.33,45.55,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,7.04,92898.0,-9312.0,102699.0,0.36,-9.11,110.55,91.37,208.0,-0.09,0.68,50.80,XY25,NTT,FINANCE
60,SAIL,228.00,45.95,58.0,M-MC,11.20,230748.0,5786.0,161870.0,1.74,2.57,70.15,74.53,192.0,0.04,1.69,35.65,XY24,BTT,STEEL
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.24,112440.0,11936.0,17170.0,4.04,11.88,15.27,28.96,79.0,0.70,0.82,49.88,MH,ATH,METALS
32,HINDZINC,730.22,27.88,64.0,M-LC,3.40,208408.0,3332.0,109227.0,2.78,1.62,52.41,54.89,52.0,0.03,1.53,26.11,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.73
2,50,75.09


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.66
LC    32.53
MC    23.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.40
X40      14.80
X40N     11.38
XR       11.06
XY25     10.26
AR        8.18
OX40N     7.88
X200      1.68
MH        1.63
X5K       1.53
SR        1.20
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.14
H-LC    25.92
H-MC    20.84
M-SC    13.07
M-LC     5.56
M-MC     2.66
L-SC     1.45
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-6.69,41.90
IT,12.69,-21.30,87.43
FINANCE,10.12,-18.79,70.61
MISC,6.93,-21.44,88.95
BANKS,6.32,-15.72,76.60
PAINTS,5.61,-20.60,38.58
ELECTRICAL,5.48,-10.00,48.60
INSURANCE,3.80,-6.89,46.85
AC,3.60,1.54,14.95


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3231110.0,22
XR,1398184.0,14
AR,1216604.0,9
X40,826880.0,10
X40N,652348.0,9
OX40N,572634.0,10
XY25,439589.0,7
SR,258720.0,2
X5K,109227.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3819775.0,29
M-SC,1460178.0,17
H-LC,1280733.0,15
H-MC,1261576.0,15
M-LC,390685.0,4
M-MC,314056.0,2
L-SC,262771.0,3
L-MC,60963.0,1
L-LC,40282.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249085.0      6
           AR         836155.0      5
M-SC       XY24       833431.0      7
H-SC       XR         818312.0      7
H-MC       XY24       584644.0      4
H-LC       X40        532587.0      5
H-SC       X40N       320334.0      4
M-MC       XY24       314056.0      2
H-SC       OX40N      264150.0      4
           SR         258720.0      2
H-LC       X40N       245134.0      3
H-MC       X40        217597.0      4
H-LC       AR         209249.0      2
H-MC       XY25       178068.0      2
L-SC       XR         173991.0      2
M-SC       AR         171200.0      2
M-LC       XY24       170205.0      2
M-SC       OX40N      146177.0      4
           XR         129975.0      2
H-LC       XY25       118540.0      3
M-LC       XR         111253.0      1
           X5K        109227.0      1
H-MC       XR         103690.0      1
M-SC       XY25       102699.0      1
H-LC       X200        95534.0      1
L-SC       OX40N       88780.0      1
H-MC       X40N        86880.0      2
H-LC       XY24        79689.0      1
M-SC       X40         76696.0      1
H-MC       OX40N       73527.0      1
H-SC       MH          73019.0      1
L-MC       XR          60963.0      1
L-LC       XY25        40282.0      1
H-MC       MH          17170.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
